In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
import numpy as np

In [2]:
# Get configuration file that has all the api tokens
def configure2():
    load_dotenv()
    UNDOC_AUTHORIZATION = (os.getenv('UNDOC_AUTHORIZATION'))
    UNDOC_CLIENT_TOKEN = (os.getenv('UNDOC_CLIENT_TOKEN'))
    return UNDOC_AUTHORIZATION, UNDOC_CLIENT_TOKEN

In [3]:
#Grab the authorization and client token from the env
UNDOC_AUTHORIZATION,UNDOC_CLIENT_TOKEN = configure2()
print(UNDOC_AUTHORIZATION)
print(UNDOC_CLIENT_TOKEN)

BQBaj6cMa03GHYDRNgR_2PlW88c1gtU-wSAwJMxDuHd_B7AW7pLz-BzQrOEMXIquNPEDgtJEIfK4E1OZSDErFNcHTBFlxdq8weyO1Xcdc3JJ3RCSFceBkNuv2ip_tU4NwPoUY3yrRDuGfXq2D_6pe_FplTDxB2U1-hLlSj2xYjt8UIXn7ksc_J9eCn6UI6ia6uR11M_sghuPdxL14SbA1bqj7K9QuM8czxK0oyI0lwfn4_oDuDmxY5MPaRpjS1qCTkDyxHLxMdwK4WmD1HYi3WUu7FdLjUTo9kf3bmkhQRvJxWzellc2Y97FMbxBowGOh9Ux7w
AAAUVfB2bpQgavQ1A4DaHJ1cL4q6+Rbh6A0PTuR1JKwWl/2AqTBQnCRtUmJ6IP76sLQp/ePaRhn/yt46NqK90TC4K1BU5qnl+1ozZC7mSKcxQoEU3NJDV0IpyzCAOhenMz4w1GM4JsDGEKSs/gpMIpi6kdyqDG3kSrB9ix7kMLun7PzzIhNzaGmCs1QYJKGMHVnzXPpSnui5urCBIfBCezopyYbajtC661g5D5w+eqgTWpDvVqFH3gQLeHR+vI/MqO0fzjP8akY3MlxTdAe9a7VoZa0JEV7BKu/qhEOT


***

## Get the play counts of each song in the Album

#### `request_album_playcounts()`

In [4]:
def request_album_playcounts(album_ID):
    '''Search ID of albums and return response to get playcounts and track_uri'''
    url = "https://api-partner.spotify.com/pathfinder/v1/query"

    querystring = {
                "operationName":"queryAlbumTracks",
                "variables":'{\"uri\":\"spotify:album:' +album_ID+ '\",\"offset\":0,\"limit\":300}',
                "extensions":"{\"persistedQuery\":{\"version\":1,\"sha256Hash\":\"f387592b8a1d259b833237a51ed9b23d7d8ac83da78c6f4be3e6a08edef83d5b\"}}"
                }

    headers = {
    "authority": "api-partner.spotify.com",
    "accept": "application/json",
    "accept-language": "en",
    "app-platform": "WebPlayer",
    "authorization":"Bearer {}".format(UNDOC_AUTHORIZATION),
    "client-token": str(UNDOC_CLIENT_TOKEN),
    "content-type": "application/json;charset=UTF-8",
    "origin": "https://open.spotify.com",
    "referer": "https://open.spotify.com/"
}

    #prcoessing the result
    result = requests.request("GET", url, headers=headers, params=querystring)
    json_result = result.json()
    #getting each track object
    items = json_result['data']['albumUnion']['tracks']['items']
    
    # get the number of items to determine the loop size
    n = len(items)
    track_uri_list = np.empty(n, dtype=object)
    playcount_list = np.empty(n)
    for i in range(n):
        track_uri_list[i] = items[i]['track']['uri']
        playcount_list[i] = items[i]['track']['playcount']

    return track_uri_list, playcount_list

Test what `request_album_playcounts()` return

In [5]:
request_album_playcounts('5MS3MvWHJ3lOZPLiMxzOU6')

(array(['spotify:track:1bDbXMyjaUIooNwFE9wn0N',
        'spotify:track:46s57QULU02Voy0Kup6UEb',
        'spotify:track:34tz0eDhGuFErIuW3q4mPX',
        'spotify:track:0wshkEEcJUQU33RSRBb5dv',
        'spotify:track:7l2nxyx7IkBX5orhkALg0V',
        'spotify:track:2ZL7WZcjuYKi1KUDtp4kCC',
        'spotify:track:0sSRLXxknVTQDStgU1NqpY',
        'spotify:track:4Flfb4fGscN9kXPOduQLrv',
        'spotify:track:7GeTsDIc5ykNB6lORO6Cee',
        'spotify:track:2KLwPaRDOB87XOYAT2fgxh',
        'spotify:track:45pUWUeEWGoSdH6UF162G8',
        'spotify:track:410TZrK18uRjtsTunG14cl',
        'spotify:track:2ZlABhxMMa43COmZ97kKsJ',
        'spotify:track:29YW2xXlo6Pc6M0SBTCXYN',
        'spotify:track:1MTXYvJ9TSqg9x6WPCDx2n',
        'spotify:track:31bsuKDOzFGzBAoXxtnAJm'], dtype=object),
 array([3.79562414e+08, 1.20039056e+08, 1.16140056e+08, 7.11348680e+07,
        8.00879400e+07, 1.04753224e+08, 6.07136470e+07, 4.77946160e+07,
        1.01212506e+08, 1.38951290e+08, 7.73573030e+07, 4.50617070e+07,


#### `get_today_playcounts()`

In [6]:
def get_today_playcounts(album_ID):
    '''Using the undocumented API to request playcounts and return a dataframe of '''
    track_uri_list, playcount_list = request_album_playcounts(album_ID)
    df = pd.DataFrame({'track_uri':track_uri_list, 'playcount':playcount_list})
    df['date_tracked'] = pd.Timestamp.today().strftime('%Y-%m-%d')
    return df

Test what `get_today_playcounts()`

In [7]:
get_today_playcounts("5MS3MvWHJ3lOZPLiMxzOU6")

,track_uri,playcount,date_tracked
0,spotify:track:1bDbXMyjaUIooNwFE9wn0N,379562414.0,2023-03-03
1,spotify:track:46s57QULU02Voy0Kup6UEb,120039056.0,2023-03-03
2,spotify:track:34tz0eDhGuFErIuW3q4mPX,116140056.0,2023-03-03
3,spotify:track:0wshkEEcJUQU33RSRBb5dv,71134868.0,2023-03-03
4,spotify:track:7l2nxyx7IkBX5orhkALg0V,80087940.0,2023-03-03
5,spotify:track:2ZL7WZcjuYKi1KUDtp4kCC,104753224.0,2023-03-03
6,spotify:track:0sSRLXxknVTQDStgU1NqpY,60713647.0,2023-03-03
7,spotify:track:4Flfb4fGscN9kXPOduQLrv,47794616.0,2023-03-03
8,spotify:track:7GeTsDIc5ykNB6lORO6Cee,101212506.0,2023-03-03
9,spotify:track:2KLwPaRDOB87XOYAT2fgxh,138951290.0,2023-03-03


#### `get_all_playcounts_today()`

In [8]:
def get_all_playcounts_today(albums_ids):
    '''Takes in array of album_ids and loop through each one to get a dataframe to be able to append to SQL database'''
    dataframe = pd.DataFrame()
    for id in albums_ids:
        temp = get_today_playcounts(id)
        dataframe = pd.concat([dataframe,temp], axis = 0)
    dataframe  = dataframe.reset_index(drop=True)
    return dataframe

Test what `get_all_playcounts_today()` return

In [9]:
test_list = np.array(['5MS3MvWHJ3lOZPLiMxzOU6', '3cf4iSSKd8ffTncbtKljXw','3SpBlxme9WbeQdI9kx7KAV', '6OQ9gBfg5EXeNAEwGSs6jK'], dtype=object)

In [11]:
get_all_playcounts_today(test_list)

,track_uri,playcount,date_tracked
0,spotify:track:1bDbXMyjaUIooNwFE9wn0N,379562414.0,2023-03-03
1,spotify:track:46s57QULU02Voy0Kup6UEb,120039056.0,2023-03-03
2,spotify:track:34tz0eDhGuFErIuW3q4mPX,116140056.0,2023-03-03
3,spotify:track:0wshkEEcJUQU33RSRBb5dv,71134868.0,2023-03-03
4,spotify:track:7l2nxyx7IkBX5orhkALg0V,80087940.0,2023-03-03
...,...,...,...
60,spotify:track:6Kj17Afjo1OKJYpf5VzCeo,144273018.0,2023-03-03
61,spotify:track:6fLVTVaHWaEfVKfEgbkf4D,31894542.0,2023-03-03
62,spotify:track:0YkUwXxnTkeJBvt5upeEtP,58712612.0,2023-03-03
63,spotify:track:05aZ9sAU1YXndHv0FMi9iW,74002303.0,2023-03-03
